In [1]:
!pip install torchviz

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/00/be/d59db2d1d52697c6adc9eacaf50e8965b6345cc143f671e1ed068818d5cf/graphviz-0.20.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.5 MB/s eta 0:00:00
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4152 sha256=0cff242e036838c0c83c0fc7412383f5667d714a4a5626f5fdb4d9e3bb6f218e
  Stored in directory: /pfs/data5/home/kit/stud/uqqww/.cache/pip/wheels/29/65/6e/db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import sys

import support_functions
from torchviz import make_dot

sys.path.append(os.path.join(os.path.abspath(os.getcwd()), "PEGNN"))
import json
import time
import myconfig_trsage_al as myconfig
import solver_trsage_al as solver
import Dataloader_trsage_al as dl
from datetime import datetime
import torch

def make_dir(path):
    try:
        os.mkdir(path)
    except:
        pass


# rebuild the folder missed?
def build_folder_and_clean(path):
    check = os.path.exists(path)
    if check:
        pass
    else:
        os.makedirs(path)


def train(job_id, settings):
    result_sheet = []

    print("Start training...")
    list_total, list_err = solver.training(settings=settings, job_id=job_id)
    print("Start evaluation...")
    best_err, r_squared = solver.evaluate(settings=settings, job_id=job_id)

    result_sheet.append([list_total, list_err, best_err, r_squared])

    # collect wandb result into file
    rtn = {
        "best_err": sum(result_sheet[0][2]) / len(result_sheet[0][2]),
        "r_squared": sum(result_sheet[0][3]) / len(result_sheet[0][3]),
        "list_total_0": result_sheet[0][0],
        "list_err_0": result_sheet[0][1],
    }

    json_dump = json.dumps(rtn)
    with open(settings['agent_dir'] + f'/{job_id}.rtn', 'w') as fresult:
        fresult.write(json_dump)



# RuntimeError: mat1 and mat2 shapes cannot be multiplied (2974x42 and 46x256)
# problem for number of the dataset_size since i change the size into minimal size
# but if this problem occurs in ssh server then means all right
if __name__ == '__main__':
    job_id = '000002'

    print('Init...')

    settings = {
        'agent_id': '00001',
        'agent_dir': './logs',
        'origin_path': './Dataset_res250_reg4c/',

        # debug mode=>data_set
        'debug': True,
        'bp': False,

        # full_batch->batch->accumulation_steps double
        'batch': 16,
        'accumulation_steps': 128 // 16,
        'test_batch': 0,

        'es_mindelta': 0.5,

        # 'num_features_in': 14,
        'num_features_in': 2,

        'num_features_out': 1,
        'emb_hidden_dim': 256,
        
        'k': 20,
        'conv_dim': 256,


        # only in this case delete in slurm
        'seed': 1,
        'holdout': 0,

        
        'model': 'trsage5',
        'fold': 4,
        'lowest_rank': 1,

        'hp_marker': 'tuned',
        'nn_length': 3,
        'nn_hidden_dim': 32,
        'dropout_rate': 0.1,

        # for transformer
        'd_model': 32,
        'num_head': 2,

        'feedforward_dim': 128,
        'dropout': 0.1,
        'num_layers_a': 2,
        'env_features_in': 11,

        
        'transformer_dec_output': 32,
        'emb_dim': 32,
        'epoch': 3,
        'es_endure': 5,
        'nn_lr': 1e-5,
        
        'trans_lr': 1e-5,

        'embedding_dim': 256,

        'aux_task_num': 1,

        # ----MAOAL----
        'hyper_lr': 1e-6,
        'hyper_decay': 0.0,
        'hyper_interval': 20,
        'hyper_aux_loss_weight': 0.1,
        'hyper_pre': 0,

        # ----Task heads----
        'heads_nn_length': 2,
        'heads_nn_hidden_dim': 64,
        'heads_dropout_rate': 0.1,
    }

    # build working folder
    dt_string = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    coffer_slot = myconfig.coffer_path + str(job_id) + '/'

    # missed
    make_dir(coffer_slot)
    build_folder_and_clean(coffer_slot)
    settings['coffer_slot'] = coffer_slot
    settings['tgt_op'] = 'mcpm10'

    train(job_id, settings)